In [1]:
import sys
!{sys.executable} -m pip install PyAthena

In [2]:
import sys
!{sys.executable} -m pip install pytz

#### Conectar con Pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [4]:
fecha_inicio = '09-01-2020'
fecha_final = '09-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

# Query sin Group by

In [5]:
for fecha in fechas:
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    
    tabla_usuarios_distancia = f'todos_usuarios_delta_dist_{codigo_fecha}'
    tabla_distancia = 'mobility'
    tabla_distancia_dia = f'distancia_recorrida_usuarios_{codigo_fecha}'
    tabla_usuarios_pais_sin_reg = f'todos_usuarios_delta_{codigo_fecha}'
    
    query = (f'''CREATE TABLE {tabla_usuarios_distancia}  
                WITH (
    external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_usuarios/{tabla_usuarios_distancia}',
    format = 'PARQUET', 
    parquet_compression = 'SNAPPY', 
    partitioned_by = ARRAY['iso_country_code']) AS 
    SELECT t_u.caid, 
           t_u.id_type,
           t_u.n_obs,
           t_u.n_obs_noche, 
           t_u.n_obs_madrugada, 
           t_u.lat_hogar, 
           t_u.long_hogar, 
           t_u.n_obs_hogar, 
           t_u.lat_mas_visitado, 
           t_u.long_mas_visitado, 
           t_u.n_obs_mas_visitado, 
           COALESCE(CAST(t_d.dist AS double), 0) AS distancia_recorrida, 
           t_u.year, 
           t_u.month, 
           t_u.day, 
           t_u.iso_country_code 
    FROM {tabla_usuarios_pais_sin_reg} AS t_u LEFT JOIN 
         (SELECT caid, iso_country_code, dist, month, day 
           FROM {tabla_distancia} WHERE partition_0='{tabla_distancia_dia}') AS  t_d 
    ON  t_u.caid=t_d.caid 
    AND t_u.month=t_d.month 
    AND t_u.day=t_d.day 
    AND t_u.iso_country_code=t_d.iso_country_code''')
    cursor.execute(query)

901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
